<a href="https://colab.research.google.com/github/pranav-vijayananth/SOMResearch/blob/main/tensorflow_som_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#HARDWARE SPECS INFORMATION

print("### CPU INFO ###")
print("--------------------------------------------------")
!cat /proc/cpuinfo
print("### MEMORY INFO ###")
print("--------------------------------------------------")
!cat /proc/meminfo

In [ ]:
#CHECKING IF CUDA IS RUNNING

!nvidia-smi
!python3 --version

In [ ]:
#INSTALLING TENSORFLOW-SOM MODULE

!wget https://raw.githubusercontent.com/cgorman/tensorflow-som/master/tf_som.py

In [ ]:
#NEED PYTHON 3.7 FOR THIS VERSION OF TF API

!pip3 install tensorflow==1.15

In [ ]:
#IMPORTS

import numpy as np

np.set_printoptions(suppress=True)

import math
import tensorflow as tf
from tf_som import SelfOrganizingMap
import subprocess
from collections import Counter
import logging
import glob
import time

print(tf.__version__)
print(np.__version__)

In [ ]:
#FUNCTION FOR GETTING FILE IN PATH

def get_file_in_path(path):
  return path.split("/")[-1]

In [ ]:
def get_name_in_path(filename):
  return filename.split(".")[0]

In [ ]:
#GETTING ALL FILE PATHS IN THE FOLDER

master_files = glob.glob("/content/drive/MyDrive/Datasets/commonfiles/*.csv")

In [ ]:
#GETTING FILE PATHS AND NAMES

file_path = master_files[0]
filename = get_file_in_path(file_path)
name = get_name_in_path(filename) 

print(f"This is the file path: {file_path}")
print(f"Name of the file: {filename}")
print(f"Name without file format: {name}")

datafile = open(file_path)

In [ ]:
#MAKING FOLDER FOR WEIGHTS

bash_command = f"mkdir /content/drive/MyDrive/Datasets/Research/CommonFilesWeights/{name}"
subprocess.run(bash_command, shell=True)

In [ ]:
#CLEANING DATA

data = []
groundtruth = []
dataread = datafile.readline()
dataread = datafile.readline()

while dataread != "": 
  a = dataread.split(",")
  l2 = []
  for j in range(0, len(a), 1):
    if j == len(a)-1:
      groundtruth.append(a[j].strip())
    else:
      try: 
        l2.append(float(a[j]))
      except:
        l2.append(0)
  data.append(l2)
  dataread = datafile.readline()

rows = len(data)
cols = len(data[0])
num_inputs = (rows * cols)

datafile.close()

In [ ]:
#GOOGLE COLAB DOES NOT SUPPORT TO EXTEND VIEWFRAME FOR HIGH AMOUNTS OF DATA

# print(f"This is the data: {data}")
print(f"Number of cols: {cols}")
print(f"These are the number of rows {rows}")
print(f"This is the grountruth: {groundtruth}")
print(f"Number of input vectors: {num_inputs}")

#CLUSTERS
uniqueValues = Counter(groundtruth).keys()
clusters = len(uniqueValues)
print(f"The clusters in dataset: {clusters}")

In [ ]:
#CREATING THE TENSORFLOW GRAPH AND LOGS FOR THE MODEL
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
graph = tf.Graph()

with graph.as_default():
  session = tf.Session(config=tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False
  ))

  #NEURONS FOR FEATURE MAP 
  neurons = int(5*math.sqrt(rows))
  m = int(math.sqrt(neurons)+1)
  n = int(math.sqrt(neurons)+1)
  neurons = (m*n)

  #BATCH SIZE 
  batch_size = 32

  #CONVERT TO NUMPY ARRAY FOR TF DATA PIPELINE
  data = np.array(data, dtype="float32")

  #MAKING THE TENSORFLOW DATRASET PIPELINE 
  input_data = tf.data.Dataset.from_tensor_slices(data)
  input_data = input_data.repeat()
  input_data = input_data.batch(batch_size)
  iterator = input_data.make_one_shot_iterator()
  next_element = iterator.get_next()

# ONE TXT FILE = ONE WEIGHT FOR EVERY ITERATION

  for i in range(1, 30+1):
    #MAKING FILE TO STORE WEIGHTS FOR EVERY ITERATION 
    weights_file = f"{name}-{i}.txt"
    bash_command = f"touch /content/drive/MyDrive/Datasets/Research/CommonFilesWeights/{name}/{weights_file}"
    subprocess.run(bash_command, shell=True)

    #BUILDING THE SOM OBJECT
    som = SelfOrganizingMap(m=clusters, n=1, dim=cols, max_epochs=11, session=session, graph=graph, input_tensor=next_element, batch_size=batch_size, initial_learning_rate=0.1, model_name='Self-Organizing-Map', softmax_activity=True)

    #MAKING + RUNNING SESSION
    init_op = tf.global_variables_initializer()
    session.run([init_op])

    start = time.time()

    #TRAINING
    som.train(num_inputs)

    stop = time.time()

    #TIME 
    print(f"Training time: {stop - start}s")

    #WEIGHTS
    som_weights = som.output_weights
    
    #STORING WEIGHTS
    wf = open(f"content/drive/MyDrive/Datasets/Research/CommonFilesWeights/{name}/{weights_file}", "a")
    for w in som_weights:
      str_w = [str(ele) for ele in w]
      temp_w = ",".join(str_w)
      wf.write(temp_w)
      wf.write("\n")
      
    wf.close()

In [ ]:
# OUTPUT WEIGHTS #
""" 
The output_weights format satisfies the given rows and columns for the neuron inputs. 
Use the shape property to return the tuple of the given weight form 
Create a file I/O function to iterate the weights into txt files
Iteration should be over 30 and epochs should be changed to 11 for faster computation
"""

weights = som.output_weights
print(weights.shape == (neurons, cols))

In [ ]:
# SSH CONNECTION

"""
For SSH connection you need to use the UCID and the hostname which is the given server. Using this you can acess the other information given
"""